In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../input/awards.csv.gz')

/Users/invinst/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
id_cols = ["First.Name", "Last.Name",
                    "Appointed.Date", "Gender"]
conflict_cols = ['Middle.Initial', 'Birth.Year',
                 'Current.Star', 'Resignation.Date']
uid = 'awards_ID'

In [8]:
def remove_duplicates(df, cols=[]):
    if not cols:
        cols = df.columns.tolist()
    return df[~df.duplicated(subset=cols, keep=False)].sort_values(cols)

In [9]:
def keep_duplicates(df, cols):
    return df[df.duplicated(subset=cols, keep=False)].sort_values(cols)

In [10]:
def resolve_conflicts(df, id_cols, conflict_cols,
                      uid, starting_uid,
                      temp_fillna=-9999):
    out_df = pd.DataFrame()

    df.reset_index(drop=True, inplace=True)
    df.fillna(temp_fillna, inplace=True)

    group_df = df.groupby(id_cols, as_index=False)

    for key, group in group_df:
        group = group.reset_index(drop=True)
        conflicts = 0

        for col in conflict_cols:
            non_nan = group[col]
            non_nan = non_nan[(non_nan != -999) &
                              (non_nan != temp_fillna)]
            if len(non_nan.unique()) > 1:
                conflicts += 1
        if conflicts == 0:
            group.insert(0, uid, starting_uid + 1)
            starting_uid += 1
        else:
            group.insert(0, uid, starting_uid + group.index + 1)
            starting_uid += max(group.index + 1)
        out_df = out_df.append(group)

    out_df[out_df == temp_fillna] = np.nan
    return out_df

In [13]:
dfu = df[id_cols + conflict_cols].drop_duplicates()
dfu.reset_index(drop=True, inplace=True)

In [14]:
rd_df = remove_duplicates(dfu, id_cols).reset_index(drop=True)

In [15]:
rd_df.insert(0, uid, rd_df.index + 1)

In [16]:
kd_df = keep_duplicates(dfu, id_cols).reset_index(drop=True)

In [17]:
rc_df = resolve_conflicts(kd_df, id_cols, conflict_cols, uid=uid, starting_uid=max(rd_df[uid]))

In [19]:
rc_df.shape()

,awards_ID,First.Name,Last.Name,Appointed.Date,Gender,Middle.Initial,Birth.Year,Current.Star,Resignation.Date
0,26095,ANN,HERR,2002-06-11,FEMALE,M,1966,-999,2010-06-16
1,26095,ANN,HERR,2002-06-11,FEMALE,M,1966,-999,NaN
0,26096,AUDREY,JURCZYKOWSKI,1999-05-10,FEMALE,NaN,1970,-999,NaN
1,26096,AUDREY,JURCZYKOWSKI,1999-05-10,FEMALE,A,1970,8280,NaN
0,26097,DAVID,ADCOCK,2002-09-30,MALE,G,1971,6673,NaN


In [21]:
rc_df = rd_df.append(rc_df)
df2 = df.merge(rc_df, on=id_cols + conflict_cols,how='left')

In [23]:
df2[df2[uid].isnull()].shape[0]

0

In [25]:
max(df2[uid])

26126

In [27]:
len(df2[uid].drop_duplicates())

26126

In [40]:
def assign_unique_ids(df, uid, id_cols, conflict_cols=[]):
    dfu = df[id_cols + conflict_cols].drop_duplicates()
    dfu.reset_index(drop=True, inplace=True)
    if conflict_cols:
        rd_df = remove_duplicates(dfu, id_cols).reset_index(drop=True)
        rd_df.insert(0, uid, rd_df.index + 1)

        kd_df = keep_duplicates(dfu, id_cols).reset_index(drop=True)
        rc_df = resolve_conflicts(kd_df, id_cols, conflict_cols,
                                  uid=uid, starting_uid=max(rd_df[uid]))
        df = df.merge(rd_df.append(rc_df),
                      on=id_cols + conflict_cols,
                      how='left')
        assert  (df[df[uid].isnull()].shape[0] == 0),\
            print('Some unique IDs are null')
        assert (max(df[uid]) == len(df[uid].drop_duplicates())),\
            print('Unique IDs are not correctly scaled')
    else:
        dfu[uid] = dfu.index + 1
        return df.merge(dfu, on=id_cols, how='left')

In [41]:
df3 = assign_unique_ids(df, uid, id_cols, conflict_cols)

0
